#  Exercise 7: In-Context Learning with GPT-4o mini

<div style="padding:15px 20px 20px 20px;border-left:3px solid green;background-color:#e4fae4;border-radius: 20px;color:#424242;">

## **Exercise Description**
In this exercise, you will investigate in-context learning using OpenAI GPT-4o mini model. 

This exercise contains two parts.

- In the first part, you will investigate in-context learning for classification based on a natural language inference (NLI) task.
    
- In the second part, you will investigate in-context learning for generation based on a story ending generation (SEG) task.


### Table of Contents
- **[PART 1: In-Context Learning for Natural Language Inference](#1)**
    - [1.1 Compare Different Shots](#11)
    - [1.2 Effect of Neutral In-Context Examples](#12)
    - [1.3 Play with Different Verbalizers](#13)
    - [1.4 Task Instructions](#14)
- **[PART 2: In-Context Learning for Story Ending Generation](#2)**
    - [2.1 Zero-Shot Generation](#21)
    - [2.2 Few-Shot Generation](#22)
    - [2.3 Task Instructions](#23)

</div>

In [ ]:
# if you are using Google Colab, mount your drive
from google.colab import drive
drive.mount('/content/drive')
# switch to the path where you put the Exercise folder into
%cd "/content/drive/MyDrive/..."

In [1]:
!pip install numpy tqdm nltk

You also need to install our **GPT wrapper** to interact with OpenAI GPT models for free.

In [2]:
!pip install gpt_wrapper-0.2.0-py3-none-any.whl

Processing ./gpt_wrapper-0.2.0-py3-none-any.whl


Import the required packages for this exercise, including our GPT wrapper.

In [3]:
import json
import numpy as np
from tqdm import tqdm
import random
from copy import deepcopy
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score

import gpt_wrapper
from gpt_wrapper.chat import Chat

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gattimartina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/gattimartina/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gattimartina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


To facilitate reproduction, we fix a random seed here.

In [4]:
seed = 233

Set up the API access to our GPT wrapper.

In [5]:
gpt_wrapper.api_base = "http://mnlp-backend-lb-1062233132.eu-central-1.elb.amazonaws.com"
gpt_wrapper.api_key = "1067c253-7e95-42bc-9b57-fb03508f30dd"

<a name="1"></a>
## **PART 1: In-Context Learning for Natural Language Inference**
---

In this part, you are going to use the GPT-4o mini model to solve the [natural language inference (NLI)](https://towardsdatascience.com/natural-language-inference-an-overview-57c0eecf6517) task based on in-context learning. For this task, model needs to classify the relation of two given sentences (premise and hypothesis) into three classes: entailment, neutral and contradiction.

Here you can take a glance of the training data used for sampling few-shot in-context examples, and the testing data used to query GPT-4o mini language model for classification (along with the gold answers for evaluation).

In [6]:
with open("nli_classification/train_classification.json", "r") as f:
    train_samples = json.load(f)
with open("nli_classification/test_classification.json", "r") as f:
    test_data = json.load(f)

print("Training Samples:")
print(train_samples["entailment"][0])
print("\n")

print("Testing Query:")
print(test_data[0]["query"])
print("\n")

print("Gold Answer:")
print(test_data[0]["gold_answer"])

Training Samples:
I know lawyers are always dreadfully careful.
I'm well aware that lawyers are always very careful.
Answer: entailment


Testing Query:
The new rights are nice enough
Everyone really likes the newest benefits 
Answer:


Gold Answer:
neutral


In [61]:
train_samples

{'entailment': ["I know lawyers are always dreadfully careful.\nI'm well aware that lawyers are always very careful.\nAnswer: entailment",
  "Nearby, the atmospheric medieval Alfama district is Lisbon's most picturesque and fascinating area.\nThe most fascinating area in Lisbon is the Alfama district.\nAnswer: entailment",
  'Mussolini annexes Abyssinia (Ethiopia), bombs  Republican Spain\nMussolini gained new land.\nAnswer: entailment',
  "yeah yeah right right right how many hours are you taking\nYeah, tell me how many hours you're taking. \nAnswer: entailment",
  'The United States responded by deploying U.S. naval forces in the region.\nThe response from the US was sending US naval forces to the area.\nAnswer: entailment',
  'When pollsters talk about a representative sample they mean a sampling that accurately reflects the population at large.\nThey speak in regards to an accurate section of society at large.\nAnswer: entailment',
  'You can be easily suckered into just such anoth

In [62]:
test_data

[{'query': 'The new rights are nice enough\nEveryone really likes the newest benefits \nAnswer:',
  'gold_answer': 'neutral'},
 {'query': 'This site includes a list of all award winners and a searchable database of Government Executive articles.\nThe Government Executive articles housed on the website are not able to be searched.\nAnswer:',
  'gold_answer': 'contradiction'},
 {'query': "uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him\nI like him for the most part, but would still enjoy seeing someone beat him.\nAnswer:",
  'gold_answer': 'entailment'},
 {'query': "yeah i i think my favorite restaurant is always been the one closest  you know the closest as long as it's it meets the minimum criteria you know of good food\nMy favorite restaurants are always at least a hundred miles away from my house. \nAnswer:",
  'gold_answer': 'contradiction'},
 {'query': "i don't know um do you do a lot of camping\nI know

In [64]:
for query in test_data : 
    print(query)
    break

{'query': 'The new rights are nice enough\nEveryone really likes the newest benefits \nAnswer:', 'gold_answer': 'neutral'}


In [63]:
for nli_class, samples in train_samples.items():
    print(nli_class)
    print(samples)
    break

entailment
["I know lawyers are always dreadfully careful.\nI'm well aware that lawyers are always very careful.\nAnswer: entailment", "Nearby, the atmospheric medieval Alfama district is Lisbon's most picturesque and fascinating area.\nThe most fascinating area in Lisbon is the Alfama district.\nAnswer: entailment", 'Mussolini annexes Abyssinia (Ethiopia), bombs  Republican Spain\nMussolini gained new land.\nAnswer: entailment', "yeah yeah right right right how many hours are you taking\nYeah, tell me how many hours you're taking. \nAnswer: entailment", 'The United States responded by deploying U.S. naval forces in the region.\nThe response from the US was sending US naval forces to the area.\nAnswer: entailment', 'When pollsters talk about a representative sample they mean a sampling that accurately reflects the population at large.\nThey speak in regards to an accurate section of society at large.\nAnswer: entailment', 'You can be easily suckered into just such another fight.\nThere

As you know by now, language models only produce a probability distribution over the vocabulary, so how many tokens to generate and how to select each token depends on the decoding algorithm. Since we are going to use GPT-4o mini model, we should get familiar with what type of decoding parameters OpenAI API offers:

**max_tokens**: Maximum number of tokens to generate, by default generates as many as needed.

**temperature**: Sampling temperature to use, between 0.0 and 2.0, default to 1.0. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

**top_p**: Nucleus sampling factor (alternative to sampling with temperature), between 0.0 and 1.0, default to 1.0. The model randomly samples from the tokens with top_p probability mass.

**presence_penalty**: Between -2.0 and 2.0, default to 0.0. Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.

**frequency_penalty**: Between -2.0 and 2.0, default to 0.0. Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.

For NLI task, we choose a small *max_tokens* because only the first non-space token generated by the model is used as the predicted class (i.e., verbalizer).

**Question:**
Why don't we simply set it to 1?

We also change the *temperature* to 0 in order to let the model make deterministic classification decisions.

In [7]:
model_args={"max_tokens": 5, "temperature": 0.0, "top_p": 1.0, "presence_penalty": 0.0, "frequency_penalty": 0.0}

You will evaluate the model's NLI performance based on the accuracy and F1 scores on each class.

In [40]:
def evaluate_nli(predictions, gold_labels, mapping):
    
    counter = np.zeros((3, 3))  # three-class confusion matrix
    
    # calculate the confusion matrix
    for p, g in zip(predictions, gold_labels):
        pid = mapping[p]
        gid = mapping[g]
        counter[gid][pid] += 1
    
    print("Confusion Matrix:")
    print(counter)
    
    pred_sum = np.sum(counter, axis=0)  # total number of predictions on each class
    gold_sum = np.sum(counter, axis=1)  # total number of test samples (gold labels) on each class
    diag = np.diagonal(counter)  # total number of correct predictions on each class
    
    acc = np.sum(diag) / np.sum(counter)  # accuracy
    
    f1 = [0, 0, 0]
    for cid in range(3):
        precision = diag[cid] / pred_sum[cid] if pred_sum[cid] != 0 else 0  # precisions on each class
        recall = diag[cid] / gold_sum[cid] if gold_sum[cid] != 0 else 0  # recalls on each class
        f1[cid] = 2 * precision * recall / (precision + recall) if precision != 0 and recall != 0 else 0  # F1 scores on each class
    
    return acc, f1[0], f1[1], f1[2]

You will use the following function to perform GPT-4o mini inference on the NLI task based on in-context learning.

In [41]:

examples = []
for nli_class, samples in train_samples.items():
    print(nli_class)
    print(samples)
    few_shot_samples = random.sample(samples, 3)
    examples.extend(few_shot_samples)
    break 

examples

entailment
["I know lawyers are always dreadfully careful.\nI'm well aware that lawyers are always very careful.\nAnswer: entailment", "Nearby, the atmospheric medieval Alfama district is Lisbon's most picturesque and fascinating area.\nThe most fascinating area in Lisbon is the Alfama district.\nAnswer: entailment", 'Mussolini annexes Abyssinia (Ethiopia), bombs  Republican Spain\nMussolini gained new land.\nAnswer: entailment', "yeah yeah right right right how many hours are you taking\nYeah, tell me how many hours you're taking. \nAnswer: entailment", 'The United States responded by deploying U.S. naval forces in the region.\nThe response from the US was sending US naval forces to the area.\nAnswer: entailment', 'When pollsters talk about a representative sample they mean a sampling that accurately reflects the population at large.\nThey speak in regards to an accurate section of society at large.\nAnswer: entailment', 'You can be easily suckered into just such another fight.\nThere

['he was a teacher\nHe worked as a teacher\nAnswer: entailment',
 'I took her across town, to see the lab.\nMe and her went to see lab. \nAnswer: entailment',
 "It should be seen as affirming the fact of Christ's presence, but not as explaining how the change takes place.\nIt is proof that Christ exists.\nAnswer: entailment"]

In [42]:
for qid, query in enumerate(tqdm(test_data)):
    print(query)
    print([query['query']])
    break

  0%|          | 0/30 [00:00<?, ?it/s]

{'query': 'The new rights are nice enough\nEveryone really likes the newest benefits \nAnswer:', 'gold_answer': 'neutral'}
['The new rights are nice enough\nEveryone really likes the newest benefits \nAnswer:']


In [43]:
def gpt_nli(train_samples, test_data, shots, predictions, gold_answers,
             instruction=None, default_class="neutral", task_name="task"):
    
    '''
    train_samples: training data for sampling in-context examples
    test_data: testing queries (with gold labels)
    shots: number of in-context examples (shots) per class
    predictions: cache for saving the model predictions
    gold_answers: cache for saving gold answers
    instruction: additional task instruction for prompting
    default_class: default prediction class if the generated token is not among the verbalizers of three NLI classes
    task_name: task name for creating chat sessions
    '''
    
    # randomly sample in-context examples
    examples = []
    for nli_class, samples in train_samples.items():
        few_shot_samples = random.sample(samples, shots[nli_class])
        examples.extend(few_shot_samples)

    random.shuffle(examples)  # randomly shuffle sampled in-context examples
    

    for qid, query in enumerate(tqdm(test_data)):
        
        if qid < len(predictions):  # skip this query if its model prediction is already saved in cache
            continue

        # TODO: Construct the prompt using the in-context examples and the testing query
        prompt = '\n\n'.join(examples) + '\n\n' + query['query']

        # create a chat session using our GPT wrapper class Chat
        chat = Chat.create(name=task_name+"_"+str(qid))
        
        # use the created chat session to query the GPT model with the input request,
        # and get back model's output message
        response = chat.ask(prompt, instruction=instruction, model_args=model_args)
        
        # model's output text is in the attribute "content",
        # we use the first word of the generated text as the prediction
        preds = response.content.strip().split()

        if preds:
            pred = preds[0].lower()
        else:
            pred = default_class
        
        # save the prediction in cache
        if pred in train_samples.keys():
            predictions.append(pred)
        else:
            predictions.append(default_class)
        
        # save the gold answer in cache for evaluation
        gold_answers.append(query["gold_answer"])

You will run the following function to perform GPT inference and evaluation.

In [44]:
def run(train_samples, test_data, class_shots, mapping, predictions, gold_answers,
        instruction=None, default_class="neutral", task_name="none"):
    try:

        gpt_nli(train_samples, test_data, class_shots, predictions, gold_answers,
                 instruction=instruction, default_class=default_class, task_name=task_name)
        
        acc, f1_ent, f1_neu, f1_con = evaluate_nli(predictions, gold_answers, mapping)
        macro_f1 = (f1_ent + f1_neu + f1_con) / 3

        print(f'Accuracy: {acc*100:.2f}% | F1: ({f1_ent*100:.2f}%, {f1_neu*100:.2f}%, {f1_con*100:.2f}%) | Macro-F1: {macro_f1*100:.2f}%')

    except Exception as error:  # OpenAI ChatGPT endpoint may get stucked by too many queries from time to time
        print(error)


<a name="11"></a>
### **1.1 Compare Different Shots**

In this part, you will compare GPT-4o mini performances under different shots (number) of in-context examples.

#### 0-shot classification:

Do not provide any in-context learning examples to the model.

Create empty caches for saving model predictions and gold answers.

In [45]:
predictions = []
gold_answers = []

Run the inference and evaluation.

**Note:** OpenAI ChatGPT endpoint may sometimes get stucked by too many queries. If running the following cell gets stucked, just re-run it, and inference will continue from the stucked query. However, do not re-run the above cell for creating the caches, which will clear the already saved predictions.

In [46]:
random.seed(seed)
np.random.seed(seed)

class_shots = {"entailment": 0, "neutral": 0, "contradiction": 0}
mapping = {"entailment": 0, "neutral": 1, "contradiction": 2}

run(train_samples, test_data, class_shots, mapping, predictions, gold_answers, task_name="1_1_shots0")

100%|██████████| 30/30 [00:24<00:00,  1.20it/s]

Confusion Matrix:
[[ 0. 10.  0.]
 [ 0. 10.  0.]
 [ 0. 10.  0.]]
Accuracy: 33.33% | F1: (0.00%, 50.00%, 0.00%) | Macro-F1: 16.67%


#### 1-shot per class:

For each class, provide 1 in-context learning example sampled from the training data.

Clear the caches.

In [47]:
predictions = []
gold_answers = []

Re-run the inference and evaluation

In [48]:
random.seed(seed)
np.random.seed(seed)

class_shots = {"entailment": 1, "neutral": 1, "contradiction": 1}
mapping = {"entailment": 0, "neutral": 1, "contradiction": 2}

run(train_samples, test_data, class_shots, mapping, predictions, gold_answers, task_name="1_1_shots1")

100%|██████████| 30/30 [00:19<00:00,  1.54it/s]

Confusion Matrix:
[[8. 2. 0.]
 [4. 4. 2.]
 [1. 0. 9.]]
Accuracy: 70.00% | F1: (69.57%, 50.00%, 85.71%) | Macro-F1: 68.43%


#### 2-shot per class:

Try 2 in-context learning examples per class.

In [49]:
predictions = []
gold_answers = []

In [50]:
random.seed(seed)
np.random.seed(seed)

class_shots = {"entailment": 2, "neutral": 2, "contradiction": 2}
mapping = {"entailment": 0, "neutral": 1, "contradiction": 2}

run(train_samples, test_data, class_shots, mapping, predictions, gold_answers, task_name="1_1_shots2")

100%|██████████| 30/30 [00:17<00:00,  1.74it/s]

Confusion Matrix:
[[8. 2. 0.]
 [3. 6. 1.]
 [1. 0. 9.]]
Accuracy: 76.67% | F1: (72.73%, 66.67%, 90.00%) | Macro-F1: 76.46%


#### 3-shot per class:

Try 3 in-context learning examples per class.

In [51]:
predictions = []
gold_answers = []

In [52]:
random.seed(seed)
np.random.seed(seed)

class_shots = {"entailment": 3, "neutral": 3, "contradiction": 3}
mapping = {"entailment": 0, "neutral": 1, "contradiction": 2}

run(train_samples, test_data, class_shots, mapping, predictions, gold_answers, task_name="1_1_shots3")

100%|██████████| 30/30 [00:22<00:00,  1.33it/s]

Confusion Matrix:
[[9. 1. 0.]
 [3. 2. 5.]
 [1. 0. 9.]]
Accuracy: 66.67% | F1: (78.26%, 30.77%, 75.00%) | Macro-F1: 61.34%


**Questions:**

1. Can model handle well the NLI task without in-context examples for learning (i.e., under the 0-shot setting)?
2. Which class are the in-context examples most helpful for detecting? and most helpless?
3. Is the more in-context examples the better?

<a name="12"></a>
### **1.2 Effect of Neutral In-Context Examples**

Try 3-shot in-context examples on the entailment and contradictions classes, but do not provide any examples on the neutral class.

In [59]:
predictions = []
gold_answers = []

In [54]:
random.seed(seed)
np.random.seed(seed)

class_shots = {"entailment": 3, "neutral": 0, "contradiction": 3}
mapping = {"entailment": 0, "neutral": 1, "contradiction": 2}

run(train_samples, test_data, class_shots, mapping, predictions, gold_answers, task_name="1_2")

100%|██████████| 30/30 [00:21<00:00,  1.40it/s]

Confusion Matrix:
[[10.  0.  0.]
 [ 6.  1.  3.]
 [ 1.  0.  9.]]
Accuracy: 66.67% | F1: (74.07%, 18.18%, 81.82%) | Macro-F1: 58.02%


In [60]:
random.seed(seed)
np.random.seed(seed)

class_shots = {"entailment": 3, "neutral": 1, "contradiction": 3}
mapping = {"entailment": 0, "neutral": 1, "contradiction": 2}

run(train_samples, test_data, class_shots, mapping, predictions, gold_answers, task_name="1_2")

100%|██████████| 30/30 [00:21<00:00,  1.40it/s]

Confusion Matrix:
[[9. 1. 0.]
 [3. 2. 5.]
 [1. 0. 9.]]
Accuracy: 66.67% | F1: (78.26%, 30.77%, 75.00%) | Macro-F1: 61.34%


**Question:** Do you observe any difference here?

<a name="13"></a>
### **1.3 Play with Different Verbalizers**

In this part, you will try to use different verbalizers for this NLI classification task. Verbalizers are mapping functions that map the numeric class labels such as 0, 1 and 2 to human-readable class labels such "entailment", "neutral" and "contradiction". However, this mapping is by no means the only correct one and in theory, we can use any mapping we want. So, next instead of using *entailment*, *neutral* and *contradiction*, you will try the following two alternatives:

- *positive*, *unrelated* and *negative*
- *a*, *b* and *c*

Build data with the above two different verbalizers.

In [72]:
mapping_to_pun = {"entailment": "positive", "neutral": "unrelated", "contradiction": "negative"}
train_samples_pun = {"positive": [], "unrelated": [], "negative": []}
test_data_pun = []

mapping_to_abc = {"entailment": "a", "neutral": "b", "contradiction": "c"}
train_samples_abc = {"a": [], "b": [], "c": []}
test_data_abc = []

for nli_class, samples in train_samples.items():
    # TODO: Populate the new training data for the two new tasks where the NLI classes are verbalized as "positive", "unrelated", and "negative",
    # and as "a", "b", and "c", respectively
    nli_class_pun = mapping_to_pun[nli_class]
    nli_class_abc = mapping_to_abc[nli_class]
    
    for sample in samples : 
        sample_pun = " ".join(sample.split(" ")[:-1] + [nli_class_pun])
        train_samples_pun[nli_class_pun].append(sample_pun)
        
        sample_abc = " ".join(sample.split(" ")[:-1] + [nli_class_abc])
        train_samples_abc[nli_class_abc].append(sample_abc)
    
for query in test_data:
    # TODO: Populate the new testing data for the two new tasks where the gold answers are verbalized as "positive", "unrelated", and "negative",
    # and as "a", "b", and "c", respectively
    query_pun = query.copy()
    query_abc = query.copy()
    
    query_pun['gold_answer'] = mapping_to_pun[query['gold_answer']]
    query_abc['gold_answer'] = mapping_to_abc[query['gold_answer']]
    
    test_data_pun.append(query_pun)
    test_data_abc.append(query_abc)
    

You can take a glance of the processed training and testing data with different verbalizers.

Data with verbalizers *positive*, *unrelated* and *negative*

In [75]:
print("Training Samples:")
print(train_samples_pun["positive"][0])
print("\n")

print("Testing Query:")
print(test_data_pun[0]["query"])
print("\n")

print("Gold Answer:")
print(test_data_pun[0]["gold_answer"])

Training Samples:
I know lawyers are always dreadfully careful.
I'm well aware that lawyers are always very careful.
Answer: positive


Testing Query:
The new rights are nice enough
Everyone really likes the newest benefits 
Answer:


Gold Answer:
unrelated


Data with verbalizers *a*, *b* and *c*

In [76]:
print("Training Samples:")
print(train_samples_abc["a"][0])
print("\n")

print("Testing Query:")
print(test_data_abc[0]["query"])
print("\n")

print("Gold Answer:")
print(test_data_abc[0]["gold_answer"])

Training Samples:
I know lawyers are always dreadfully careful.
I'm well aware that lawyers are always very careful.
Answer: a


Testing Query:
The new rights are nice enough
Everyone really likes the newest benefits 
Answer:


Gold Answer:
b


#### Re-do the classification with new verbalizers.

Try verbalizers *positive*, *unrelated* and *negative* under the 2-shot setting.

In [77]:
predictions = []
gold_answers = []

In [78]:
random.seed(seed)
np.random.seed(seed)

class_shots_pun = {"positive": 2, "unrelated": 2, "negative": 2}
mapping_pun = {"positive": 0, "unrelated": 1, "negative": 2}

run(train_samples_pun, test_data_pun, class_shots_pun, mapping_pun,
    predictions, gold_answers, default_class="unrelated", task_name="1_3_pun")

100%|██████████| 30/30 [01:17<00:00,  2.60s/it]

Confusion Matrix:
[[8. 0. 2.]
 [4. 1. 5.]
 [1. 0. 9.]]
Accuracy: 60.00% | F1: (69.57%, 18.18%, 69.23%) | Macro-F1: 52.33%


Try verbalizers *a*, *b* and *c* under the 2-shot setting in 1.1

In [79]:
predictions = []
gold_answers = []

In [80]:
random.seed(seed)
np.random.seed(seed)

class_shots_abc = {"a": 2, "b": 2, "c": 2}
mapping_abc = {"a": 0, "b": 1, "c": 2}

run(train_samples_abc, test_data_abc, class_shots_abc, mapping_abc,
    predictions, gold_answers, default_class="b", task_name="1_3_abc")

100%|██████████| 30/30 [00:54<00:00,  1.82s/it]

Confusion Matrix:
[[6. 3. 1.]
 [1. 4. 5.]
 [1. 0. 9.]]
Accuracy: 63.33% | F1: (66.67%, 47.06%, 72.00%) | Macro-F1: 61.91%


**Questions:**

1. Are new verbalizers better or worse than the original ones?
2. What do you think could be the reason?

<a name="14"></a>
### **1.4 Task Instructions**

So far, we have only provided the model with a few examples of the NLI task, however, we didn't tell the model explicitly what the task is! But we also know that these models are self-supervised with a lot of data to follow human instructions to solve tasks such as "Translate English to French" or "Answer in the style of Shakespeare". 

In this part, you will try to come up with high-level task instructions/descriptions that can help the model understand the NLI task better and improve its performance.

In [82]:
predictions = []
gold_answers = []

In [83]:
random.seed(seed)
np.random.seed(seed)

# TODO: Come up with a simple task description for the NLI task. You can also consider adding a role-playing element to the task description.
instruction = "Guess whether the second statement is entailed by the first statement, contradicts the first statement, or is neutral to the first statement."
class_shots = {"entailment": 1, "neutral": 1, "contradiction": 1}
mapping = {"entailment": 0, "neutral": 1, "contradiction": 2}

run(train_samples, test_data, class_shots, mapping,
    predictions, gold_answers, instruction=instruction, task_name="1_4_intro1")

100%|██████████| 30/30 [00:26<00:00,  1.15it/s]

Confusion Matrix:
[[8. 2. 0.]
 [4. 6. 0.]
 [1. 0. 9.]]
Accuracy: 76.67% | F1: (69.57%, 66.67%, 94.74%) | Macro-F1: 76.99%


**Question:** What kind of instruction help improve the model performance and why?

<a name="2"></a>
## **PART 2: In-Context Learning for Story Ending Generation**
---

In this part, you will switch to using the GPT-4o mini model to solve the story ending generation (SEG) task based on in-context learning. For this task, model is given four lines of story plot and needs to generate the fifth line of the story plot as an ending.

You can take a glance of the training data used for sampling few-shot in-context examples, and the testing data used to query GPT-4o mini language model for story completion (along with the reference story ending).

In [84]:
with open("story_generation/train_generation.json", "r") as f:
    train_samples_sg = json.load(f)
with open("story_generation/test_generation.json", "r") as f:
    test_data_sg = json.load(f)

print("Training Samples:")
print(train_samples_sg[0])
print("\n")

print("Testing Query:")
print(test_data_sg[0]["query"])
print("\n")

print("Reference Story Ending:")
print(test_data_sg[0]["reference_ending"])

Training Samples:
Dan's parents were overweight.
Dan was overweight as well.
The doctors told his parents it was unhealthy.
His parents understood and decided to make a change.
Output: They got themselves and Dan on a diet.


Testing Query:
A few days ago I decided to take my dog Sable for a walk.
She is a half-pit bull half-bulldog with a lot of strength.
After I got her leash on I opened the garage to head outside.
She tried bolting out of the garage and dragged me along with her.
Output:


Reference Story Ending:
After the half hour walk my muscles were sore as if I had worked out.


We will remove the limit on the maximum number of tokens since we are dealing with an open-ended text generation.

We also change the *temperature* and *top_p* to 0.7 and 0.95 in order to enable the model's creativity and make it generate more diverse story endings.

In [85]:
model_args = {"temperature": 0.7, "top_p": 0.95, "presence_penalty": 0.0, "frequency_penalty": 0.0}

You will evaluate the model generation performance based on [METEOR](https://aclanthology.org/W05-0909.pdf). This metric is originally proposed to evaluate machine translation quality, but later widely used in evaluating open-domain text (e.g., dialogues and stories) generation. It measures the alignments (i.e., matches) between words in the hypothesis to reference, by sequentially applying exact match, stemmed match and wordnet based synonym match.

In [86]:
def evaluate_seg(generation, reference):
    ref_tokens = word_tokenize(reference)
    gen_tokens = word_tokenize(generation)
    score = meteor_score([ref_tokens], gen_tokens)
    return score

You will use the following function to perform GPT-4o mini generation on the SEG task based on in-context learning.

In [95]:
def gpt_seg(train_samples, test_data, shot, generations, queries, reference_endings,
             instruction=None, task_name="task"):

    '''
    train_samples: training data for sampling in-context examples
    test_data: testing queries (with reference story endings)
    shot: number of in-context examples
    generations: cache for saving the model generations
    queries: cache for saving the input queries (i.e., four-line stories to be completed)
    reference_endings: cache for reference story endings
    instruction: additional task instruction for prompting
    task_name: task name for creating chat sessions
    '''
    
    # randomly sample in-context examples and shuffle them
    examples = random.sample(train_samples, shot)
    random.shuffle(examples)

    for qid, query in enumerate(tqdm(test_data)):
        
        if qid < len(generations):  # skip this query if its model generated story ending is already saved in cache
            continue

        # TODO: Construct the prompt using the in-context examples and the testing query
        prompt = '\n'.join(examples + [query['query']])

        # create a chat session using our GPT wrapper and query the model to get the story ending generation
        chat = Chat.create(name=task_name+"_"+str(qid))
        message = chat.ask(prompt, instruction=instruction, model_args=model_args)
        
        # save the model generation, story query and reference ending in caches
        generations.append(message.content)
        queries.append(query["query"])
        reference_endings.append(query["reference_ending"])

You will run the following function to perform GPT-4o mini generation and evaluation.

In [96]:
def run(train_samples, test_data, shot, generations, queries, reference_endings, instruction=None, task_name="none"):
    
    try:
        
        gpt_seg(train_samples, test_data, shot,
                 generations, queries, reference_endings,
                 instruction=instruction, task_name=task_name)

        meteor_scores = []
        print()

        for qid, query in enumerate(queries):

            meteor = evaluate_seg(generations[qid], reference_endings[qid])
            print("Query "+str(qid+1)+f' METEOR Score: {meteor*100:.2f}') 

            meteor_scores.append(meteor)

        meteor_avg = sum(meteor_scores)/len(meteor_scores)
        print(f'Average METEOR Score: {meteor_avg*100:.2f}')
    
    except Exception as error:  # OpenAI ChatGPT endpoint may get stucked by too many queries from time to time
        
        print(error)


<a name="21"></a>
### **2.1 Zero-Shot Generation**

Try 0-shot story ending generation (i.e., without any in-context examples).

Create caches for saving model predictions, queries and reference story endings.

In [97]:
generations_21 = []
queries_21 = []
reference_endings_21 = []

Run the generation and evaluation.

**Note:** Similar to Part 1, re-run the cell if it gets stucked.

In [98]:
random.seed(seed)
np.random.seed(seed)

run(train_samples_sg, test_data_sg, 0, generations_21, queries_21, reference_endings_21, task_name="2_1_shots0")

100%|██████████| 10/10 [00:34<00:00,  3.49s/it]



Query 1 METEOR Score: 14.32
Query 2 METEOR Score: 28.96
Query 3 METEOR Score: 34.28
Query 4 METEOR Score: 16.05
Query 5 METEOR Score: 17.18
Query 6 METEOR Score: 17.50
Query 7 METEOR Score: 11.59
Query 8 METEOR Score: 17.07
Query 9 METEOR Score: 20.46
Query 10 METEOR Score: 19.08
Average METEOR Score: 19.65


You can print the saved caches and compare the quality of the reference and model-generated story endings.

In [99]:
print("Queries:\n"+queries_21[0])
print("GPT-4o mini Generation:\n"+generations_21[0])
print("Reference:\n"+reference_endings_21[0])

Queries:
A few days ago I decided to take my dog Sable for a walk.
She is a half-pit bull half-bulldog with a lot of strength.
After I got her leash on I opened the garage to head outside.
She tried bolting out of the garage and dragged me along with her.
Output:
GPT-4o mini Generation:
As soon as I opened the garage door, Sable's excitement was palpable. With her powerful build, she lunged forward, and before I knew it, I was being pulled along behind her. I stumbled for a moment, trying to regain my footing as her enthusiasm propelled us both into the open air. 

"Whoa, Sable! Easy there!" I called out, my heart racing. She was ready to explore the world beyond the confines of our home, and I knew I had to get a handle on her excitement. 

I took a deep breath, planted my feet firmly on the ground, and gently pulled back on the leash, trying to redirect her energy. "Let’s take it slow," I said, hoping to calm her down a bit. After a moment of tugging, she paused, glancing back at me 

<a name="22"></a>
### **2.2 Few-Shot Generation**

Try adding 5-shot in-context examples for this generation task.

In [100]:
generations_22 = []
queries_22 = []
reference_endings_22 = []

In [101]:
random.seed(seed)
np.random.seed(seed)

run(train_samples_sg, test_data_sg, 5, generations_22, queries_22, reference_endings_22, task_name="2_2_shots5")

100%|██████████| 10/10 [00:29<00:00,  2.97s/it]


Query 1 METEOR Score: 9.62
Query 2 METEOR Score: 18.87
Query 3 METEOR Score: 5.21
Query 4 METEOR Score: 7.41
Query 5 METEOR Score: 47.70
Query 6 METEOR Score: 60.13
Query 7 METEOR Score: 9.43
Query 8 METEOR Score: 18.79
Query 9 METEOR Score: 10.10
Query 10 METEOR Score: 14.85
Average METEOR Score: 20.21


**Question:** Do few-shot examples improve the model's story ending generation quality?

You can print the saved caches and make more comparisons between the model generations in 2.1 and 2.2.

In [102]:
print("Queries:\n"+queries_22[0])
print("GPT-4o mini Generation:\n"+generations_22[0])
print("Reference:\n"+reference_endings_22[0])

Queries:
A few days ago I decided to take my dog Sable for a walk.
She is a half-pit bull half-bulldog with a lot of strength.
After I got her leash on I opened the garage to head outside.
She tried bolting out of the garage and dragged me along with her.
Output:
GPT-4o mini Generation:
I quickly regained my balance and managed to pull her back.
Reference:
After the half hour walk my muscles were sore as if I had worked out.


<a name="23"></a>
### **2.3 Add Instructions**

Try 1-shot in-context learning with overall task instruction.

In [103]:
generations_23 = []
queries_23 = []
reference_endings_23 = []

In [104]:
random.seed(seed)
np.random.seed(seed)

# TODO: Come up with a simple task description for the story generation task. You can also consider adding a role-playing element to the task deåscription.
instruction = "You are a creative storyteller. Generate an ending of the given story."
run(train_samples_sg, test_data_sg, 1, generations_23, queries_23, reference_endings_23,
    instruction=instruction, task_name="2_3_intro")

100%|██████████| 10/10 [01:20<00:00,  8.00s/it]


Query 1 METEOR Score: 10.60
Query 2 METEOR Score: 6.12
Query 3 METEOR Score: 12.07
Query 4 METEOR Score: 13.10
Query 5 METEOR Score: 8.95
Query 6 METEOR Score: 12.64
Query 7 METEOR Score: 9.45
Query 8 METEOR Score: 10.12
Query 9 METEOR Score: 11.63
Query 10 METEOR Score: 5.42
Average METEOR Score: 10.01


**Question:** Does overall task instruction help improve the model's 1-shot story ending generation quality?

You can print the saved caches and make more comparisons between the model generations in 2.1 and 2.3.

In [105]:
print("Queries:\n"+queries_23[0])
print("GPT-4o mini Generation:\n"+generations_23[0])
print("Reference:\n"+reference_endings_23[0])

Queries:
A few days ago I decided to take my dog Sable for a walk.
She is a half-pit bull half-bulldog with a lot of strength.
After I got her leash on I opened the garage to head outside.
She tried bolting out of the garage and dragged me along with her.
Output:
GPT-4o mini Generation:
As we burst out into the fresh air, Sable's energy was palpable. She darted ahead, her strong legs propelling her forward with an enthusiasm that was both thrilling and slightly terrifying. I stumbled behind her, laughing and trying to regain my balance. 

"Whoa there, girl!" I called out, but she was too busy sniffing every blade of grass and chasing after fluttering leaves. It was as if the entire world had come alive for her, and I was just along for the ride. 

After a few minutes of her zigzagging through the neighborhood, I finally managed to reel her in a bit, guiding us towards the nearby park. The sun was shining, and children were playing, their laughter ringing out like music in the warm air.

**Question**: Depending on your instruction, do you observe any undesirable pattern in the model generations when 0 or 1 shot examples are given? How can we avoid this pattern in model generations?

In [106]:
generations_23 = []
queries_23 = []
reference_endings_23 = []

In [107]:
random.seed(seed)
np.random.seed(seed)

instruction = "You are a creative storyteller. Generate a one-sentence ending of the given story."
run(train_samples_sg, test_data_sg, 1, generations_23, queries_23, reference_endings_23,
    instruction=instruction, task_name="2_3_intro")

100%|██████████| 10/10 [00:29<00:00,  2.98s/it]


Query 1 METEOR Score: 10.99
Query 2 METEOR Score: 27.40
Query 3 METEOR Score: 12.71
Query 4 METEOR Score: 23.85
Query 5 METEOR Score: 17.61
Query 6 METEOR Score: 25.25
Query 7 METEOR Score: 10.79
Query 8 METEOR Score: 16.13
Query 9 METEOR Score: 11.81
Query 10 METEOR Score: 12.00
Average METEOR Score: 16.85


<a name="23"></a>
### **Further Reading**

In this exercise session, we played with different prompting techniques for in-context learning, however, all our methods were quite basic. In the past few years, the research community has developed various sophisticated prompting approaches that have been shown to elicit more from large language models. This field has grown so much that [*prompt engineering*](https://www.techtarget.com/searchenterpriseai/definition/AI-prompt-engineer) has become a legit job position. Some well-known prompting techniques are [Chain-of-thought](https://arxiv.org/abs/2201.11903), [Tree-of-thought](https://arxiv.org/abs/2305.10601), [Metacognitive prompting](https://arxiv.org/abs/2308.05342), [Self-consistency](https://arxiv.org/abs/2203.11171) and etc. You can find more information about all these techniques in this [prompting guide](https://www.promptingguide.ai). We encourage you to play with these techniques and compare your results.